In [8]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from imblearn.over_sampling import SMOTE


# Load Data

In [1]:
import os
import numpy as np


def load_data(folder):
    data = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file.endswith('.npy'):
                    # Load numpy array
                    array = np.load(file_path).flatten()
                    data.append(array)
                    # Label phishing as 1, benign as 0
                    label = 1 if (folder.find('phishing') != -1) else 0
                    labels.append(label)
    return np.array(data), np.array(labels)

# Đường dẫn đến thư mục chứa dữ liệu
phishing_path = 'VisualPhish/phishing_features'
benign_path = 'VisualPhish/trusted_list_features'

# Load dữ liệu
phishing_data, phishing_labels = load_data(phishing_path)
benign_data, benign_labels = load_data(benign_path)



In [2]:
print(phishing_data.shape)
print(phishing_labels.shape)
print(benign_data.shape)
print(benign_labels.shape)

(1195, 512)
(1195,)
(9363, 512)
(9363,)


In [3]:
from sklearn.model_selection import train_test_split

def split_and_remove(data, labels):
    # Tách 20% dữ liệu cho test set và lưu chỉ mục
    train_idx, test_idx = train_test_split(
        np.arange(len(labels)), test_size=0.2, random_state=42, stratify=labels)

    # Phân chia dữ liệu dựa trên chỉ mục
    test_set = data[test_idx]
    test_labels = labels[test_idx]
    train_set = np.delete(data, test_idx, axis=0)
    train_labels = np.delete(labels, test_idx, axis=0)
    return train_set, train_labels, test_set, test_labels

# Áp dụng cho dữ liệu phishing và benign
phishing_train, phishing_labels_train, phishing_test, phishing_labels_test = split_and_remove(phishing_data, phishing_labels)
benign_train, benign_labels_train, benign_test, benign_labels_test = split_and_remove(benign_data, benign_labels)

# Gộp dữ liệu huấn luyện và kiểm thử
train_data = np.concatenate((phishing_train, benign_train))
train_labels = np.concatenate((phishing_labels_train, benign_labels_train))
test_data = np.concatenate((phishing_test, benign_test))
test_labels = np.concatenate((phishing_labels_test, benign_labels_test))


In [4]:
print(phishing_train.shape)
print(phishing_labels_train.shape)
print(phishing_test.shape)
print(phishing_labels_test.shape)
print(benign_train.shape)
print(benign_labels_train.shape)
print(benign_test.shape)
print(benign_labels_test.shape)
print('======')
print(test_data.shape)
print(test_labels.shape)

(956, 512)
(956,)
(239, 512)
(239,)
(7490, 512)
(7490,)
(1873, 512)
(1873,)
(2112, 512)
(2112,)


In [9]:
import matplotlib.pyplot as plt
import numpy as np

def showFeature(features):
  image = features.reshape(16, 32)
  #print(image.shape)
  #print(image)
  
  plt.subplot(1, 1, 1)  # Tạo subplot
  plt.imshow(image, cmap='viridis')  # Sử dụng viridis làm bảng màu
  plt.title(f'Feature map')
  plt.axis('off')  # Ẩn trục tọa độ

  plt.show()

# DNN

In [5]:

def create_model():
    DNN_model = Sequential([
        Dense(1024, activation='relu', input_shape=(512,)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return DNN_model

In [11]:
# Chia dữ liệu thành 10 phần

federated_data = []

# Giả sử X, y là dữ liệu của bạn
num_clients = 1

num_samples_phishing_per_client = len(phishing_train) // num_clients
num_samples_benign_per_client = len(benign_train) // num_clients

for i in range(num_clients):
    start_idx = i * num_samples_phishing_per_client
    end_idx = (i + 1) * num_samples_phishing_per_client if i != num_clients - 1 else len(phishing_train)
    
    start_idx_benign = i * num_samples_benign_per_client
    end_idx_benign = (i + 1) * num_samples_benign_per_client if i != num_clients - 1 else len(benign_train)

    X_client_phishing = phishing_train[start_idx:end_idx]
    y_client_phishing = phishing_labels_train[start_idx:end_idx]

    X_client_benign = benign_train[start_idx_benign:end_idx_benign]
    y_client_benign = benign_labels_train[start_idx_benign:end_idx_benign]

    print("Client ", i, ": ")
    smote = SMOTE()
    X_client, y_client = smote.fit_resample(np.concatenate((X_client_phishing, X_client_benign), axis=0), 
                                            np.concatenate((y_client_phishing, y_client_benign), axis=0))
    print(X_client.shape)
    print(y_client.shape)
    federated_data.append((X_client, y_client))



Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f8f942ac160>
Traceback (most recent call last):
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Client  0 : 
(14980, 512)
(14980,)


In [12]:
def preprocessing(X_data, y_data):
    # Chuyển đổi sang TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data))
    dataset = dataset.batch(64) # BatchSize
    return dataset

In [13]:
federated_data_train = [preprocessing(X_client, y_client) for X_client, y_client in federated_data]

In [14]:
def has_weights_changed(old_weights, new_weights, tolerance=1e-5):
    """
    Kiểm tra xem trọng số của mô hình có thay đổi giữa các vòng đào tạo hay không.
    :param old_weights: Trọng số của mô hình từ vòng trước.
    :param new_weights: Trọng số mới của mô hình.
    :param tolerance: Ngưỡng dung sai cho sự thay đổi.
    :return: True nếu có sự thay đổi, ngược lại False.
    """
    for old_layer, new_layer in zip(old_weights, new_weights):
        if np.any(np.abs(new_layer - old_layer) > tolerance):
            return True
    return False

In [15]:
print(len(test_data))

2112


In [16]:
# Tạo tập dữ liệu từ test_data và test_labels
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

# Chia thành các batch (ví dụ: batch_size=32)
test_dataset = test_dataset.batch(64)

In [17]:
model = create_model()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

# Tạo một callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',   # Giám sát mất mát của tập kiểm định
    min_delta=0.0000001,      # Thay đổi tối thiểu để coi là cải thiện
    patience=10,          # Số lượng epochs không cải thiện trước khi dừng
    verbose=1,            # Hiển thị thông báo khi dừng
    mode='min',           # Dừng quá trình huấn luyện khi giá trị 'val_loss' không giảm
    restore_best_weights=True # Khôi phục trọng số tốt nhất khi kết thúc
)

# Giả sử 'model' là mô hình của bạn và bạn đã chuẩn bị dữ liệu
# model = ...

# Huấn luyện mô hình với dữ liệu và callback EarlyStopping
history = model.fit(
    federated_data_train[0],
    validation_data=test_dataset,
    epochs=1000,          # Số lượng epochs tối đa
    callbacks=[early_stopping]
)
# In số lượng epochs đã thực hiện
print("Số lượng epochs đã thực hiện:", len(history.history['loss']))


Epoch 1/1000
  7/235 [..............................] - ETA: 4s - loss: 0.6465 - accuracy: 0.9866

235/235 [==============================] - 5s 20ms/step - loss: 0.6647 - accuracy: 0.6029 - val_loss: 0.7889 - val_accuracy: 0.1132
Epoch 2/1000
235/235 [==============================] - 5s 19ms/step - loss: 0.7176 - accuracy: 0.5835 - val_loss: 0.7823 - val_accuracy: 0.1132
Epoch 3/1000
235/235 [==============================] - 5s 19ms/step - loss: 0.7047 - accuracy: 0.5007 - val_loss: 0.7611 - val_accuracy: 0.1136
Epoch 4/1000
235/235 [==============================] - 5s 20ms/step - loss: 0.7009 - accuracy: 0.5009 - val_loss: 0.7529 - val_accuracy: 0.1136
Epoch 5/1000
235/235 [==============================] - 5s 19ms/step - loss: 0.6998 - accuracy: 0.5011 - val_loss: 0.7473 - val_accuracy: 0.1136
Epoch 6/1000
235/235 [==============================] - 5s 19ms/step - loss: 0.6991 - accuracy: 0.5013 - val_loss: 0.7427 - val_accuracy: 0.1141
Epoch 7/1000
235/235 [==============================] - 5s 20ms/step - loss: 0.6038 - accuracy: 0.8268 - val_loss: 21.7319 - val_accuracy: 0.11

In [19]:
model.evaluate(test_dataset)
# test_data => wrong

33/33 [==============================] - 0s 6ms/step - loss: 0.7514 - accuracy: 0.1132


[0.7513643503189087, 0.11316287517547607]

In [20]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score

def evaluate_model(model, test_dataset, loss_fn):
    # Đánh giá mô hình trên tập dữ liệu kiểm tra
    test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)

    # Dự đoán nhãn và tính toán các chỉ số
    y_true = np.concatenate([y_batch.numpy().flatten() for _, y_batch in test_dataset])
    y_pred = np.concatenate([model.predict(x_batch, verbose=0).flatten() for x_batch, _ in test_dataset])

    y_pred_rounded = np.array([1 if y > 0.5 else 0 for y in y_pred])

    print(len(y_true))
    print(len(y_pred))
    cnt0 = 0
    cnt1 = 0
    for y in y_pred_rounded:
        if y == 0:
            cnt0 += 1
        else:
            cnt1 += 1
    print(cnt0)
    print(cnt1)

    # Tính toán các chỉ số đánh giá
    f1 = f1_score(y_true, y_pred_rounded)
    recall = recall_score(y_true, y_pred_rounded)
    precision = precision_score(y_true, y_pred_rounded)
    auc = roc_auc_score(y_true, y_pred_rounded)

    return test_loss, test_accuracy, f1, recall, precision, auc


In [25]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
test_loss, test_accuracy, f1, recall, precision, auc = evaluate_model(model, test_dataset, loss_fn)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test F1: {f1}")
print(f"Test Recall: {recall}")
print(f"Test Precision: {precision}")
print(f"Test Auc: {auc}")

2112
2112
27
2085
Test Loss: 0.7288432121276855
Test Accuracy: 0.125
Test F1: 0.20481927710843373
Test Recall: 0.99581589958159
Test Precision: 0.11414868105515588
Test Auc: 0.5048486865766999


In [40]:

def train_model_on_client(model, client_dataset, epochs=1):
    for epoch in range(epochs):
        print("e:",epoch)
        for step, (x_batch_train, y_batch_train) in enumerate(client_dataset):
            loss, accuracy = model.train_on_batch(x_batch_train, y_batch_train)
        print(f'Loss: {loss}, Accuracy: {accuracy}')
    
    return model.get_weights(), loss, accuracy
    
def aggregate_weights(client_weights):
    new_weights = [np.mean([client_weights[i][layer] for i in range(len(client_weights))], axis=0) for layer in range(len(client_weights[0]))]
    return new_weights

# Training loop
global_model = create_model()

global_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

previous_weights = global_model.get_weights()

for round_num in range(1):
    print("/////////////////////////////ROUND ", round_num + 1, "///////////////////////////")
    client_weights = []
    losses = []
    accuracies = []

    for client_data in federated_data_train:
        weights, loss, accuracy = train_model_on_client(global_model, client_data, epochs=1)
        client_weights.append(weights)
        losses.append(loss)
        accuracies.append(accuracy)
        if has_weights_changed(weights, previous_weights):
            print("Client i weight thay doi")
        else:
            print("Client i weight khong doi")
    
    # Aggregate the weights and update the global model
    averaged_weights = aggregate_weights(client_weights)
    global_model.set_weights(averaged_weights)
    

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    test_loss, test_accuracy, f1, recall, precision, auc = evaluate_model(global_model, test_dataset, loss_fn)

    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")
    print(f"Test F1: {f1}")
    print(f"Test Recall: {recall}")
    print(f"Test Precision: {precision}")
    print(f"Test Auc: {auc}")

    if has_weights_changed(previous_weights, averaged_weights):
        print(f"Trọng số thay đổi sau vòng {round_num + 1}")
    else:
        print(f"Không có sự thay đổi trong trọng số sau vòng {round_num + 1}")

    # Cập nhật trọng số cho vòng tiếp theo
    
    previous_weights = averaged_weights
    
    # Evaluate the model on the test data
    print("res")
    
    global_model.evaluate(test_dataset)
    res = global_model.predict(test_dataset).flatten()
    print(res)

    


/////////////////////////////ROUND  1 ///////////////////////////
e: 0
Loss: 0.0, Accuracy: 1.0
Client i weight thay doi
2112
2112
Test Loss: 20.750492095947266
Test Accuracy: 0.8868371248245239
Test F1: 0.0
Test Recall: 0.0
Test Precision: 0.0
Test Auc: 0.5
Trọng số thay đổi sau vòng 1
res
24/33 [====================>.........] - ETA: 0s - loss: 28.5319 - accuracy: 0.8444   

/home/haohao/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


33/33 [==============================] - 0s 6ms/step
[0. 0. 0. ... 0. 0. 0.]
